# The Spinning Effective One-Body Initial Condition Solver

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body initial condition solver as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.  That is, we follow Section IV A of [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067).

### Throughout this module, we will refer to
* [Pan, Buonanno, Buchman et. al. (2010)](https://arxiv.org/abs/0912.3466v2) as PBB2010,
* [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) as BCD2006, and
* [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows, matching the "steps" listed in :

0. [Introduction](#intro)
1. [Step 1:](#step1) Rotate $\hat{\bf L}_{N} \to {\bf e}_{z}$ and ${\bf N}_{0} \to {\bf e}_{x}$
1. [Step 2:](#step2) Compute ${\bf r}$, ${\bf p}_{r}$, ${\bf p}_{\theta}$, and ${\bf p}_{\phi}$
1. [Step 3:](#step3) Rotate $\hat{\bf L} \to {\bf e}_{z}$
1. [Step 4:](#step4) Compute $\dot{\bf r}$
1. [Step 5:](#step5) Invert the rotation of Step 3
1. [Step 6:](#step6) Invert the rotation of Step 1
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two compact objects (e.g. black holes or neutron stars) with masses $m_{1}$, $m_{2}$ and spins ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (see [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.69)) describes the dynamics of this system.  We seek initial conditions for nonadiabatic evolutions of such a system, and follow [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Section IV A.

Please note that throughout this notebook we adpot the following conventions:
1. $c = G = 1$ where $c$ is the speed of light in a vacuum and $G$ is Newton's gravitational constant, and
1. hatted vectors (e.g. $\hat{\bf L}_{N}$) usually denote scaled or unit vectors.

We start with the following parameters:
1. binary parameters $m_{1}$, $m_{2}$, ${\bf S}_{1}$, and ${\bf S}_{2}$,
1. system inclination $\iota$ and initial frequency $f$, and
1. SEOB model parameters (constants dependent on $m_{1}$, $m_{2}$.

Our goal is to produce initial dynamical variables
1. ${\bf x} = \left( x, y, z \right)$, and
1. ${\bf p} = \left( p_{x}, p_{y}, p_{z} \right)$.

We choose a spatial coordinate basis $\left\{ {\bf e}_{1}, {\bf e}_{2}, {\bf e}_{3} \right\}$ so that the initial separation vector ${\bf r}$ between the compact objects lies in the ${\bf e}_{1},{\bf e}_{3}$-plane.

We include below the physical parameters necessary to compute the initial conditions.  Solving for the initial conditions requires computing $\frac{ \partial H_{\rm real} }{ \partial p_{i} }$ for $i \in \left\{ x, y, z \right\}$, <font color='red'>which we defer to another module</font>.

Please note that in [BCD2006](https://arxiv.org/abs/gr-qc/0508067) the initial conditions are solved for given an initial separation; here we use a given initial frequency instead.  The difference is that we solve Equation (4.8) rather than (4.9).

LALSuite (LALSimIMRSpinPrecEOB.c Line 1283) sets the inclination to zero for "comparison with C++ and NR".  <font color='red'>Is this really necessary?</font>

<font color='red'>Note about $\gamma$.</font>

<font color='red'>Note about $\mathcal{M}_{\odot}$, geometrized solar mass.</font>

In [1]:
import sympy as sp
m1,m2,S1x,S1y,S1z,S2x,S2y,S2z,f,inc = sp.symbols("m1 m2 S1x S1y S1z S2x S2y S2z f inc",real=True)
partHpartpx,partHpartpy,partHpartpz = sp.symbols("partHpartpx partHpartpy partHpartpz",real=True)

gamma = 0.577215664901532860606512090082402431
Msol = 4.92549095e-6

<a id='step1'></a>

# Step 1: Rotate $\hat{\bf L}_{N} \to {\bf e}_{z}$ and ${\bf N}_{0} \to {\bf e}_{x}$ \[Back to [top](#toc)\]
$$\label{step1}$$

<a id='massterms'></a>

## Mass and spin terms \[Back to [top](#toc)\]
$$\label{massterms}$$

We begin by defining the following useful quantities:

\begin{align*}
    M &= m_{1} + m_{2} \\
    \eta &= \frac{ m_{1} m_{2} }{ M^{2} } \\
    \hat{\bf S}_{1} &= \frac{ 1 }{ M^{2} } {\bf S}_{1} \\
    \hat{\bf S}_{2} &= \frac{ 1 }{ M^{2} } {\bf S}_{2}
\end{align*}

In [2]:
M = m1 + m2
Msq = M*M
eta = m1*m2/Msq
S1hatx = S1x/Msq
S1haty = S1y/Msq
S1hatz = S1z/Msq
S2hatx = S2x/Msq
S2haty = S2y/Msq
S2hatz = S2z/Msq

<a id='ln'></a>

## Orbital Angular Momenutm $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{ln}$$

Since we assume that the initial separation vector ${\bf r}$ between $m_{1}$ and $m_{2}$ lies in the ${\bf e}_{1},{\bf e}_{3}$-plane, the inital orbital anbular momentum vector ${\bf L}_{N}$ is given by

\begin{align*}
    \hat{\bf L}_{N}^{x} &= \sin \left( \iota \right) \\
    \hat{\bf L}_{N}^{y} &= 0.0 \\
    \hat{\bf L}_{N}^{z} &= \cos \left( \iota \right)
\end{align*}

<font color='red'>${\bf L}_{N}$ is computed at LALSimIMRSpinEOBInitialConditionsPrec.c Lines 789-791.  Line 787 says "taken from STPN code"; is this supposed to constitute a reference?  This appears to be (Euclidean) geometry... Furthermore, we hard-code $\iota = 0$ before the function XLALSimIMRSpinEOBInitialConditionsPrec is called (see LALSimIMRSpinPrecEOB.c Lines 1283 and 1311).  Thus it is unclear that we need to actually compute these values (rather than just setting ${\bf L}_{N} = \left( 0, 0, 1 \right)$.</font>

In [3]:
LNhatx = sp.sin(inc)
LNhaty = 0.0
LNhatz = sp.cos(inc)

<a id='rhat'></a>

## Position $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rhat}$$

From LALSimIMRSpinEOBInitialConditionsPrec.c Lines 803-805, we have

\begin{align*}
    \hat{\bf r}^{x} &= \sin \left( \arctan \left( - \frac{ \hat{\bf L}_{N}^{z} }{ \hat{\bf L}_{N}^{x} } \right) \right) \\
    \hat{\bf r}^{y} &= 0.0 \\
    \hat{\bf r}^{z} &= \cos \left( \arctan \left( - \frac{ \hat{\bf L}_{N}^{z} }{ \hat{\bf L}_{N}^{x} } \right) \right)
\end{align*}

<font color='red'>To avoid a singularity, the code includes a check for $\hat{\bf L}_{N}^{z} > 0.9999$.  This is always the case when $\iota = 0$, and in that case
\begin{align*}
    \hat{\bf r}^{x} &= 1 \\
    \hat{\bf r}^{y} &= 0 \\
    \hat{\bf r}^{z} &= 0.
\end{align*}
</font>

In [4]:
theta0 = sp.atan(-LNhatz / LNhatx)
rhatx = sp.sin(theta0)
rhaty = 0.0
rhatz = sp.cos(theta0)

<a id='vhat'></a>

## Velocity $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{vhat}$$

From LALSimIMRSpinEOBInitialConditionsPrec.c Lines 809-811, we have

\begin{equation*}
    \hat{\bf v} = \hat{\bf L}_{N} \times \hat{\bf r}.
\end{equation*}

<font color='red'></font>

In [5]:
vhatx = LNhaty*rhatz - LNhatz*rhaty
vhaty = LNhatz*rhatx - LNhatx*rhatz
vhatz = LNhatx*rhaty - LNhaty*rhatx

<a id='rotationmatrix'></a>

## Rotation matrix \[Back to [top](#toc)\]
$$\label{rotationmatrix}$$

From LALSimIMRSpinEOBInitialConditionsPrec.c Lines 135--143, the rotation matrix from the $\left\{ {\bf e}_{x}, {\bf e}_{y}, {\bf e}_{z} \right\}$ frame to the $\left\{ \hat{\bf r}, \hat{\bf v}, \hat{\bf L}_{N} \right\}$ is given by

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}.
\end{equation*}

<font color='red'>The matrix to invert the rotation is simply the transpose:</font>

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf v}^{x} & \hat{\bf L}_{N}^{x} \\
        \hat{\bf r}^{y} & \hat{\bf v}^{y} & \hat{\bf L}_{N}^{y} \\
        \hat{\bf r}^{z} & \hat{\bf v}^{z} & \hat{\bf L}_{N}^{z}\end{bmatrix}.
\end{equation*}

<font color='red'>There seems to be a bunch of wasted code in LALSimIMRSpinEOBInitialConditionsPrec.c Lines 96--118.  The computed angles are not used...</font>

In [6]:
rotate00 = rhatx
rotate01 = rhaty
rotate02 = rhatz
rotate10 = vhatx
rotate11 = vhaty
rotate12 = vhatz
rotate20 = LNhatx
rotate21 = LNhaty
rotate22 = LNhatz

invert00 = rhatx
invert01 = vhatx
invert02 = LNhatx
invert10 = rhaty
invert11 = vhaty
invert12 = LNhaty
invert20 = rhatz
invert21 = vhatz
invert22 = LNhatz

<a id='rotaterhat'></a>

## Rotate $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rotaterhat}$$

We rotate $\hat{\bf r}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 843.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf r}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf r}^{x} \\ \hat{\bf r}^{y} \\ \hat{\bf r}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [7]:
rhatprmx = rhatx*rhatx + rhaty*rhaty + rhatz*rhatz
rhatprmy = vhatx*rhatx + vhaty*rhaty + vhatz*rhatz
rhatprnz = LNhatx*rhatx + LNhaty*rhaty + LNhatz*rhatz

<a id='rotatevhat'></a>

## Rotate $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{rotatevhat}$$

We rotate $\hat{\bf v}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 844.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf v}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf v}^{x} \\ \hat{\bf v}^{y} \\ \hat{\bf v}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [8]:
vhatprmx = rhatx*vhatx + rhaty*vhaty + rhatz*vhatz
vhatprmy = vhatx*vhatx + vhaty*vhaty + vhatz*vhatz
vhatprmz = LNhatx*vhatx + LNhaty*vhaty + LNhatz*vhatz

<a id='rotatelnhat'></a>

## Rotate $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{rotatelnhat}$$

We rotate $\hat{\bf L}_{N}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 845.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf L}_{N}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf L}_{N}^{x} \\ \hat{\bf L}_{N}^{y} \\ \hat{\bf L}_{N}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [9]:
LNhatprmx = rhatx*LNhatx + rhaty*LNhaty + rhatz*LNhatz
LNhatprmy = vhatx*LNhatx + vhaty*LNhaty + vhatz*LNhatz
LNhatprmz = LNhatx*LNhatx + LNhaty*LNhaty + LNhatz*LNhatz

<a id='rotates1'></a>

## Rotate ${\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotates1}$$

We rotate ${\bf S}_{1}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 846.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{1}^{x} \\ {\bf S}_{1}^{y} \\ {\bf S}_{1}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [10]:
S1prmx = rhatx*S1x + rhaty*S1y + rhatz*S1z
S1prmy = vhatx*S1x + vhaty*S1y + vhatz*S1z
S1prmz = LNhatx*S1x + LNhaty*S1y + LNhatz*S1z

<a id='rotates2'></a>

## Rotate ${\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotates2}$$

We rotate ${\bf S}_{2}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 847.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{2}^{x} \\ {\bf S}_{2}^{y} \\ {\bf S}_{2}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [11]:
S2prmx = rhatx*S2x + rhaty*S2y + rhatz*S2z
S2prmy = vhatx*S2x + vhaty*S2y + vhatz*S2z
S2prmz = LNhatx*S2x + LNhaty*S2y + LNhatz*S2z

<a id='rotates1hat'></a>

## Rotate $\hat{\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotates1hat}$$

We rotate $\hat{\bf S}_{1}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 846.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{1}^{x} \\ \hat{\bf S}_{1}^{y} \\ \hat{\bf S}_{1}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [12]:
S1hatprmx = rhatx*S1hatx + rhaty*S1haty + rhatz*S1hatz
S1hatprmy = vhatx*S1hatx + vhaty*S1haty + vhatz*S1hatz
S1hatprmz = LNhatx*S1hatx + LNhaty*S1haty + LNhatz*S1hatz

<a id='rotates2hat'></a>

## Rotate $\hat{\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotates2hat}$$

We rotate $\hat{\bf S}_{2}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 846.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{2}^{x} \\ \hat{\bf S}_{2}^{y} \\ \hat{\bf S}_{2}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [13]:
S2hatprmx = rhatx*S2hatx + rhaty*S2haty + rhatz*S2hatz
S2hatprmy = vhatx*S2hatx + vhaty*S2haty + vhatz*S2hatz
S2hatprmz = LNhatx*S2hatx + LNhaty*S2haty + LNhatz*S2hatz

<a id='step2'></a>

# Step 2: Compute ${\bf r}$, ${\bf p}_{r}$, ${\bf p}_{\theta}$, and ${\bf p}_{\phi}$ \[Back to [top](#toc)\]
$$\label{step2}$$

<a id='omega'></a>

## $\omega$ \[Back to [top](#toc)\]
$$\label{omega}$$

<font color='red'>In appropriate units, the inital orbital frequency is</font>

\begin{equation*}
    \omega = M \mathcal{M}_{\odot} \pi f.
\end{equation*}

<font color='red'></font>

In [14]:
omega = sp.pi*Msol*M*f

<a id='velocity'></a>

## Initial Velocity $v_{0}$ \[Back to [top](#toc)\]
$$\label{velocity}$$

<font color='red'>In appropriate units, the inital velocity is</font>

\begin{equation*}
    v_{0} = \sqrt[3]{ \omega }.
\end{equation*}

<font color='red'></font>

<a id='step3'></a>

# Step 3: Rotate $\hat{\bf L} \to {\bf e}_{z}$ \[Back to [top](#toc)\]
$$\label{step3}$$

<a id='step4'></a>

# Step 4: Compute $\dot{\bf r}$ \[Back to [top](#toc)\]
$$\label{step4}$$

<a id='step5'></a>

# Step 5: Invert the rotation of Step 3 \[Back to [top](#toc)\]
$$\label{step5}$$

<a id='step6'></a>

# Step 6: Invert the rotation of Step 1 \[Back to [top](#toc)\]
$$\label{step6}$$

<a id='skerr'></a>

# ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{skerr}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.2), (5.63), and (5.67) we have

\begin{equation*}
    {\bf S}_{\rm Kerr} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation*}

Taking the square of [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.9) <font color='red'>(be careful with the factor or $M$...)</font>,

\begin{equation*}
    a^{2} = \frac{ {\bf S}_{Kerr} \cdot {\bf S}_{Kerr} }{ M^{2} }
\end{equation*}

so that

\begin{equation*}
    a = \sqrt{ a^{2} }.
\end{equation*}

In [15]:
SKerr1 = S1x + S2x
SKerr2 = S1y + S2y
SKerr3 = S1z + S2z
asq = (SKerr1*SKerr1 + SKerr2*SKerr2 + SKerr3*SKerr3)/(M*M)
a = sp.sqrt(asq)

<a id='w'></a>

# $\varpi$ \[Back to [top](#toc)\]
$$\label{w}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.7)

\begin{equation*}
    \varpi^{2} = r^{2} + a^{2}.
\end{equation*}

In [16]:
w2 = r*r + asq

NameError: global name 'r' is not defined

<a id='Dinv'></a>

# $D^{-1}$ \[Back to [top](#toc)\]
$$\label{Dinv}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.82)

\begin{equation*}
    D^{-1} = 1 + \log \left[ 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3} \right].
\end{equation*}

In [ ]:
Dinv = 1 + sp.log(1 + 6*eta*u*u + 2*(26 - 3*eta)*eta*u*u*u)

<a id='k'></a>

# $K$ \[Back to [top](#toc)\]
$$\label{k}$$

<font color='red'>We see $K$ used ("KK") in LALSpinPrecHcapRvecDerivative_v3opt.c Line 65, before it's calculuated on Line 86.</font>

Following the discussion in the two paragrahs after [BB2010](https://arxiv.org/abs/0912.3517) Equation (6.11), we have

\begin{equation*}
    K = 1.4467 \left( 1 - 4 \eta \right)^{2} + 4 \left( 1 - 2 \eta \right) \eta.
\end{equation*}

In [ ]:
K = 1.4467*(1 - 4*eta)*(1 - 4*eta) + 4*(1 - 2*eta)*eta

<a id='deltaubar'></a>

# $\bar{\Delta}_{u}$ \[Back to [top](#toc)\]
$$\label{deltaubar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.75)

\begin{equation*}
    \bar{\Delta}_{u} = \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 2 u }{ \eta K -1 } + \frac{ 1 }{ \left( \eta K - 1 \right)^{2} }.
\end{equation*}

In [ ]:
Deltaubar = asq*u*u/(M*M) + 2*u/(eta*K - 1) + 1/((eta*K - 1)*(eta*K - 1))

<a id='calibcoeffs'></a>

# Calibration Coefficients $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ \[Back to [top](#toc)\]
$$\label{calibcoeffs}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.77)--(5.81), we have

\begin{align*}
    \Delta_{0} &= K \left( \eta K - 2 \right) \\
    \Delta_{1} &= -2 \left( \eta K - 1 \right) \left( K + \Delta_{0} \right) \\
    \Delta_{2} &= \frac{1}{2} \Delta_{1} \left( -4 \eta K + \Delta_{1} + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{0}\\
    \Delta_{3} &= \frac{1}{3} \left[ -\Delta_{1}^{3} + 3 \left( \eta K - 1 \right) \Delta_{1}^{2} + 3 \Delta_{2} \Delta_{1} - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_{2} + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{1} \right] \\
    \Delta_{4} &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_{1}^{2} - 2 \Delta_{2} \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_{1}^{4} - 8 \left( \eta K - 1 \right) \Delta_{1}^{3} - 12 \Delta_{2} \Delta_{1}^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_{2} + \Delta_{3} \right] \Delta_{1} \right.\\
        &\left.\ \ \ \ \ + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_{2}^{2} - 4 \Delta_{3} \left( \eta K - 1 \right) \right] \right\}
\end{align*}

<font color='red'>There are two more coefficients in the code that are not listed in [BB2010](https://arxiv.org/abs/0912.3517), and whose origin I have not been able to trace.</font>

\begin{align*}
    \Delta_{5} &= \\
    \Delta_{5l} &= 
\end{align*}

In [ ]:
Delta0 = K*(eta*K - 2)
Delta1 = -2*(eta*K - 1)*(K + Delta0)
Delta2 = sp.Rational(1,2)*Delta1*(-4*eta*K + Delta1 + 4) - a*a*(eta*K - 1)*(eta*K - 1)*Delta0/(M*M)
Delta3 = sp.Rational(1,3)*(-Delta1**3 + 3*(eta*K - 1)*Delta1**2 + 3*Delta2*Delta1 - 6*(eta*K - 1)*(-eta*K + Delta2 + 1)
                           - 3*a*a*(eta*K - 1)**2*Delta1/(M*M))
Delta4 = sp.Rational(1,12)*(6*a*a*(Delta1**2 - 2*Delta2)*(eta*K - 1)**2/(M*M) + 3*Delta1**4 - 8*(eta*K - 1)*Delta1**3
                           -12*Delta2*Delta1**2 + 12*(2*(eta*K - 1)*Delta2 + Delta3)*Delta1
                           + 12*(sp.Rational(94,3) - sp.Rational(41,32)*sp.pi**2)*(eta*K - 1)**2
                           + 6*(Delta2**2 - 4*Delta3*(eta*K - 1)))
Delta5 = (eta*K - 1)**2*(-sp.Rational(4237,60) + sp.Rational(128,5)*gamma
                          + 2275*sp.pi*sp.pi*sp.Rational(1,512)
                          - sp.Rational(1,3)*a*a*(Delta1**3 - 3*Delta1*Delta2 + 3*Delta3)
                          - (Delta1**5 - 5*Delta1**3*Delta2 + 5*Delta1*Delta2**2 + 5*Delta1**2*Delta3
                             - 5*Delta2*Delta3 - 5*Delta1*Delta4)*sp.Rational(1,5)/((eta*K - 1)**2)
                          + (Delta1**4 - 4*Delta1**2*Delta2 + 2*Delta2**2 + 4*Delta1*Delta3 - 4*Delta4)
                          *sp.Rational(1,2)/(eta*K - 1) + sp.Rational(256,5)*sp.log(2))
Delta5l = sp.Rational(64,5)*(eta*K - 1)**2

<a id='deltau'></a>

# $\Delta_{u}$ \[Back to [top](#toc)\]
$$\label{deltau}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.73)

\begin{equation*}
    \Delta_{u} = \bar{\Delta}_{u} \left[ 1 + \eta \Delta_{0} + \eta \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) \right].
\end{equation*}

<font color='red'>Note that in LALSpinPrechcapRvecDerivative_v3opt.c line 77 we take an absolute value.</font>

In [ ]:
Deltau = Deltaubar*(1 + eta*Delta0 + eta*sp.log(1 + Delta1*u + Delta2*u*u + Delta3*u*u*u + Delta4*u*u*u*u))

<a id='deltat'></a>

# $\Delta_{t}$ \[Back to [top](#toc)\]
$$\label{deltat}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.71)

\begin{equation*}
    \Delta_{t} = r^{2} \Delta_{u}.
\end{equation*}

In [ ]:
Deltat = r*r*Deltau

<a id='deltar'></a>

# $\Delta_{r}$ \[Back to [top](#toc)\]
$$\label{deltar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.38)

\begin{equation*}
    \Delta_{r} = \Delta_{t} D^{-1}.
\end{equation*}

In [ ]:
Deltar = Deltat*Dinv

<a id='xi'></a>

# $\xi$ \[Back to [top](#toc)\]
$$\label{xi}$$

Inverting [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (28),

\begin{equation*}
    \xi = \frac{ \sqrt{ \Delta_{t} \Delta_{r} } }{ \varpi^{2} }.
\end{equation*}

In [ ]:
xi = sp.sqrt(Deltat*Deltar)/w2

<a id='constants'></a>

# Constants \[Back to [top](#toc)\]
$$\label{constants}$$

<font color='red'>We now move to LALSimIMRCalculateSpinPrecEOBHCoeffs.c to recompute the coefficients.  In Lines 77--84, the following are listed as "constants are fits taken from PRD 86, 024011 (2012) E. 37".  That [reference](https://arxiv.org/abs/1202.0790) does not, however, contain either that many constants or that many significant digits...  On Lines 98 and 99, we see coefficients b3 and bb3 set to zero.  There is a note to check whether b3 and bb3 can ever be nonzero; if that question was answered, there is no record of it.</font>

Below, we will denote c0 by $c_{0}$, and similarly for the other constants.

In [ ]:
c0  = 1.4467
c1  = -1.7152360250654402
c2  = -3.246255899738242
c20  = 1.712
c21  = -1.803949138004582
c22  = -39.77229225266885
c23  = 103.16588921239249

b3 = 0.
bb3 = 0.0

<a id='k2'></a>

# $K$ (redo) \[Back to [top](#toc)\]
$$\label{k2}$$

<font color='red'>We see $K$ used ("KK") in LALSpinPrecHcapRvecDerivative_v3opt.c Line 65, before it's calculuated on Line 86.  It is NOT clear that the computations coincid; the question of WHY this recomputation takes place is open.</font>

We now re-compute $K$ (which makes no sense, because it's a constant dependent only on $\eta$).  Since v3 always takes SpinAlignedEOBversion = 2, on LALSpinPrecHcapRvecDerivative_v3opt.c Line 102 we have

\begin{equation*}
    K = c_{20} + c_{21} \eta + c_{22} \eta^{2} + c_{23} \eta^{3}.
\end{equation*}

In [ ]:
K = c20 + c21*eta + c22*eta*eta + c23*eta*eta*eta

<a id='calibcoeffs2'></a>

# Calibration Coefficients $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ (redo) \[Back to [top](#toc)\]
$$\label{calibcoeffs2}$$

<font color='red'>We see these coefficients used ("ki") in LALSpinPrecHcapRvecDerivative_v3opt.c Line 71, before they're calculuated on Line 86.  It IS clear that these computations agree with the previous computations; the question of WHY this recomputation takes place is open.</font>

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.77)--(5.81), we have

\begin{align*}
    \Delta_{0} &= K \left( \eta K - 2 \right) \\
    \Delta_{1} &= -2 \left( \eta K - 1 \right) \left( K + \Delta_{0} \right) \\
    \Delta_{2} &= \frac{1}{2} \Delta_{1} \left( -4 \eta K + \Delta_{1} + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{0}\\
    \Delta_{3} &= \frac{1}{3} \left[ -\Delta_{1}^{3} + 3 \left( \eta K - 1 \right) \Delta_{1}^{2} + 3 \Delta_{2} \Delta_{1} - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_{2} + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{1} \right] \\
    \Delta_{4} &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_{1}^{2} - 2 \Delta_{2} \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_{1}^{4} - 8 \left( \eta K - 1 \right) \Delta_{1}^{3} - 12 \Delta_{2} \Delta_{1}^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_{2} + \Delta_{3} \right] \Delta_{1} \right.\\
        &\left.\ \ \ \ \ + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_{2}^{2} - 4 \Delta_{3} \left( \eta K - 1 \right) \right] \right\}
\end{align*}

<font color='red'>There are two more coefficients in the code that are not listed in [BB2010](https://arxiv.org/abs/0912.3517), and whose origin I have not been able to trace.</font>

\begin{align*}
    \Delta_{5} &= \\
    \Delta_{5l} &= 
\end{align*}

In [ ]:
Delta0 = K*(eta*K - 2)
Delta1 = -2*(eta*K - 1)*(K + Delta0)
Delta2 = sp.Rational(1,2)*Delta1*(-4*eta*K + Delta1 + 4) - a*a*(eta*K - 1)*(eta*K - 1)*Delta0/(M*M)
Delta3 = sp.Rational(1,3)*(-Delta1**3 + 3*(eta*K - 1)*Delta1**2 + 3*Delta2*Delta1 - 6*(eta*K - 1)*(-eta*K + Delta2 + 1)
                           - 3*a*a*(eta*K - 1)**2*Delta1/(M*M))
Delta4 = sp.Rational(1,12)*(6*a*a*(Delta1**2 - 2*Delta2)*(eta*K - 1)**2/(M*M) + 3*Delta1**4 - 8*(eta*K - 1)*Delta1**3
                           -12*Delta2*Delta1**2 + 12*(2*(eta*K - 1)*Delta2 + Delta3)*Delta1
                           + 12*(sp.Rational(94,3) - sp.Rational(41,32)*sp.pi**2)*(eta*K - 1)**2
                           + 6*(Delta2**2 - 4*Delta3*(eta*K - 1)))
Delta5 = (eta*K - 1)**2*(-sp.Rational(4237,60) + sp.Rational(128,5)*gamma
                          + 2275*sp.pi*sp.pi*sp.Rational(1,512)
                          - sp.Rational(1,3)*a*a*(Delta1**3 - 3*Delta1*Delta2 + 3*Delta3)
                          - (Delta1**5 - 5*Delta1**3*Delta2 + 5*Delta1*Delta2**2 + 5*Delta1**2*Delta3
                             - 5*Delta2*Delta3 - 5*Delta1*Delta4)*sp.Rational(1,5)/((eta*K - 1)**2)
                          + (Delta1**4 - 4*Delta1**2*Delta2 + 2*Delta2**2 + 4*Delta1*Delta3 - 4*Delta4)
                          *sp.Rational(1,2)/(eta*K - 1) + sp.Rational(256,5)*sp.log(2))
Delta5l = sp.Rational(64,5)*(eta*K - 1)**2

<a id='calibconsts'></a>

# Calibration Constants \[Back to [top](#toc)\]
$$\label{calibconst}$$

<font color='red'>From LALSimIMRCalculateSpinPrecEOBHCoeffs.c Lines 127--130 we have</font>

\begin{align*}
    d_{1} &= -69.5 \\
    d_{1v2} &= -74.71 - 156 \eta + 627.5 \eta^{2} \\
    d_{H_{\rm eff}\ \rm SS} &= 2.75 \\
    d_{H_{\rm eff}\ \rm SSv2} &= 8.127 - 154.2 \eta + 830.8 \eta^{2}.
\end{align*}

<font color='red'>This marks the end of the CalculateSpinPrecEOB routine.</font>

In [ ]:
d1 = -69.5
d1v2 = -sp.Rational(7471,100) - 156*eta + sp.Rational(1255,2)*eta*eta
dheffSS = sp.Rational(11,4)
dheffSSv2 = sp.Rational(8127,1000) - sp.Rational(771,5)*eta + sp.Rational(4154,5)*eta*eta

<a id='tortoise_matrix'></a>

# Tortoise Conversion Matrix \[Back to [top](#toc)\]
$$\label{tortoise_matrix}$$

<font color='red'>We're now back to LALSpinPrecHcapRvecDerivative_v3opt.c, Lines 92--96.</font>

From [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (A3) the matrix for the coordinate conversion to tortoise coordinates is

\begin{align*}
    \begin{pmatrix} 1 + \frac{ x^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & \frac{ x z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ y^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x z }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ z^{2} }{ r^{2} } \left( \xi - 1 \right) \end{pmatrix}
\end{align*}

In [ ]:
ximinus1 = xi - 1
toTort = sp.Array([[1 + x*x*ximinus1/(r*r), x*y*ximinus1/(r*r), x*z*ximinus1/(r*r)],
                   [x*y*ximinus1/(r*r), 1 + y*y*ximinus1/(r*r), y*z*ximinus1/(r*r)],
                   [x*z*ximinus1/(r*r), y*z*ximinus1/(r*r), 1 + z*z*ximinus1/(r*r)]])

# At this point we compute partial derivatives of $H_{\rm real}$ with respect to the momentum coordinates

<a id='sigma'></a>

# $\boldsymbol{\sigma}$ \[Back to [top](#toc)\]
$$\label{sigma}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.2),

\begin{equation*}
    \boldsymbol{\sigma} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation*}

In [ ]:
sigma1 = S1x + S2x
sigma2 = S1y + S2y
sigma3 = S1z + S2z

<a id='sigmastar'></a>

# $\boldsymbol{\sigma}^{*}$ \[Back to [top](#toc)\]
$$\label{sigmastar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.3),

\begin{equation*}
    \boldsymbol{\sigma}^{*} = \frac{ m_{2} }{ m_{1} } {\bf S}_{1} + \frac{ m_{1} }{ m_{2} }{\bf S}_{2}.
\end{equation*}

In [ ]:
sigmastar1 = S1x*m2/m1 + S2x*m1/m2
sigmastar2 = S1y*m2/m1 + S2y*m1/m2
sigmastar3 = S1z*m2/m1 + S2z*m1/m2

<a id='updatecoeffs'></a>

# Update Coeffs \[Back to [top](#toc)\]
$$\label{updatecoeffs}$$

<a id='e3'></a>

## ${\bf e}_{3}$ \[Back to [top](#toc)\]
$$\label{e3}$$

From LALSimIMRSpinPrecEOBHcapExactDerivative.c Line 104, we only compute ${\bf e}_{3}$ this way if $a \not= 0$.

From the discussion in [BB2010](https://arxiv.org/abs/0912.3517) after equation (5.52),

\begin{equation*}
    {\bf e}_{3} = {\bf S}_{\rm Kerr}.
\end{equation*}

<font color='red'>From LALSimIMRSpinPrecEOBHcapExactDerivative.c Lines 107--109,

\begin{equation*}
    {\bf e}_{3} = \frac{ {\bf S}_{\rm Kerr} }{ a }.
\end{equation*}
</font>

In [ ]:
e31 = SKerr1/a
e32 = SKerr2/a
e33 = SKerr3/a

<a id='scaleds'></a>

## Scaled ${\bf S}$ \[Back to [top](#toc)\]
$$\label{scaleds}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Line 114,

\begin{equation*}
    {\bf S}_{\rm scaled} = M^{2} {\bf S}.
\end{equation*}

In [ ]:
S1scaled1 = S1x*M*M
S1scaled2 = S1y*M*M
S1scaled3 = S1z*M*M
S2scaled1 = S2x*M*M
S2scaled2 = S2y*M*M
S2scaled3 = S2z*M*M

<a id='rcrossrdot'></a>

## ${\bf r} \times \dot{\bf r}$ \[Back to [top](#toc)\]
$$\label{rcrossrdot}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 119--121,

\begin{equation*}
    {\bf r} \times \dot{\bf r} = {\bf x} \times \frac{ \partial {\bf H}_{\rm real} }{ {\bf p} }
\end{equation*}

Ultimately we want the *scaled* ${\bf r} \times \dot{\bf r}$, whic we denote by ${\bf L}_{N}$.

In [ ]:
rcrossrdot1 = y*partHpartp3 - z*partHpartp2
rcrossrdot2 = z*partHpartp1 - x*partHpartp3
rcrossrdot3 = x*partHpartp2 - y*partHpartp1
rcrossrdotmag = sp.sqrt(rcrossrdot1*rcrossrdot1
                        + rcrossrdot2*rcrossrdot2 + rcrossrdot3*rcrossrdot3)

<a id='ln'></a>

## ${\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{ln}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 123--125,

\begin{equation*}
    {\bf L}_{N} = \frac{ {\bf r} \times \dot{\bf r} }{ \left\lvert {\bf r} \times \dot{\bf r} \right\rvert }.
\end{equation*}

In [ ]:
LN1 = rcrossrdot1/rcrossrdotmag
LN2 = rcrossrdot2/rcrossrdotmag
LN3 = rcrossrdot3/rcrossrdotmag

<a id='s1dotln'></a>

## ${\bf S}_{1} \cdot {\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{s1dotln}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 126,

\begin{align*}
    {\bf S}_{1} \cdot {\bf L}_{N} = {\bf S}_{1}^{x} {\bf L}_{N}^{1} + {\bf S}_{1}^{y} {\bf L}_{N}^{2} + {\bf S}_{1}^{z} {\bf L}_{N}^{3}.
\end{align*}

In [ ]:
S1dotLN = S1x*LN1 + S1y*LN2 + S1z*LN3

<a id='s2dotln'></a>

## ${\bf S}_{2} \cdot {\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{s2dotln}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 127,

\begin{align*}
    {\bf S}_{2} \cdot {\bf L}_{N} = {\bf S}_{2}^{x} {\bf L}_{N}^{1} + {\bf S}_{2}^{y} {\bf L}_{N}^{2} + {\bf S}_{2}^{z} {\bf L}_{N}^{3}.
\end{align*}

In [ ]:
S2dotLN = S2x*LN1 + S2y*LN2 + S2z*LN3

<a id='chis'></a>

## $\chi_{\rm S}$ \[Back to [top](#toc)\]
$$\label{chis}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 128,

\begin{equation*}
    \chi_{\rm S} = \frac{ 1 }{ 2 } \left( {\bf S}_{1} \cdot {\bf L}_{N} + {\bf S}_{2} \cdot {\bf L}_{N} \right).
\end{equation*}

In [ ]:
chiS = 0.5 * (s1dotLN + s2dotLN)

<a id='chia'></a>

## $\chi_{\rm A}$ \[Back to [top](#toc)\]
$$\label{chia}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 129,

\begin{equation*}
    \chi_{\rm A} = \frac{ 1 }{ 2 } \left( {\bf S}_{1} \cdot {\bf L}_{N} - {\bf S}_{2} \cdot {\bf L}_{N} \right).
\end{equation*}

In [ ]:
chiA = 0.5 * (s1dotLN - s2dotLN)

<a id='chikerr'></a>

## $\chi_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{chikerr}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 131 and [Taracchini, Pan, Buonanno, et. al. (2012)](https://arxiv.org/abs/1202.0790) Equation (31) (note $\nu$ of [Taracchini, Pan, Buonanno, et. al. (2012)](https://arxiv.org/abs/1202.0790) is $\eta$ of [BB2010](https://arxiv.org/abs/0912.3517)),

\begin{align*}
    \chi_{\rm Kerr} &= \left( 1 - 2 \eta \right) \chi_{\rm S} + \sqrt{ 1 - 4 \eta } \chi_{\rm A} \\
        &= \left( 1 - 2 \eta \right) \chi_{\rm S} + \frac{ m_{1} - m_{2} }{ m_{1} + m_{2} } \chi_{\rm A}
\end{align*}

In [ ]:
chiKerr = (1 - 2*eta)*chiS + (m1 - m2)/(m1 + m2)*chiA

# CalcSpinPrecFacWaveformCoefficients

<a id='convertderiv'></a>

## Convert derivatives \[Back to [top](#toc)\]
$$\label{convertderiv}$$

From LALSpinPrecHcapRvecDerivative_v3opt.c Lines 141,

\begin{equation*}
    \dot{\bf r} = \begin{bmatrix} 1 + \frac{ x^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & \frac{ x z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ y^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x z }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ z^{2} }{ r^{2} } \left( \xi - 1 \right) \end{bmatrix} \begin{bmatrix} p_{x} \\ p_{y} \\ p_{z} \end{bmatrix}
\end{equation*}

In [ ]:
rdot1 = toTort[0]*px + toTort[1]*py + toTort[2]*pz
rdot2 = toTort[3]*px + toTort[4]*py + toTort[5]*pz
rdot3 = toTort[6]*px + toTort[7]*py + toTort[8]*pz

<a id='rcrossrdot'></a>

# ${\bf r} \times \dot{\bf r}$ \[Back to [top](#toc)\]
$$\label{rcrossrdot}$$

This is computed at LALSimIMRSpinPrecEOB.c Line 1379.  Note this is a bit different than our previous rendition; why?  And why are we recomputing this here?

In [ ]:
rcrossrdot1 = y*rdot3 - z*rdot2
rcrossrdot2 = z*rdot1 - x*rdot3
rcrossrdot3 = x*rdot2 - y*rdot1

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Initial_Conditions.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!rm -f Tut*.out Tut*.aux Tut*.log